<a href="https://colab.research.google.com/github/desbaa32/Master2BD_tp_pro/blob/master/TpP_Securite_Chiffrement_cle_publique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Chiffrement cle publique***

In [41]:
import random
import math
from sympy import isprime, primerange
import hashlib

#### *I) Protocole de Diffie-Hellman*

In [42]:

# Données perso
N = 23010677
jour_naissance = 13
mois_naissance = 2
annee_naissance = 1997

print("===_ I) DIFFIE-HELLMAN _===")
print(f"Numéro étudiant N = {N}")

# 1)  nombre premier p le plus proche de N
def trouver_premier_proche(n):
    # Chercher dans les deux directions
    candidats = []
    i = 0
    while len(candidats) < 2:
        if isprime(n + i):
            candidats.append(n + i)
        if isprime(n - i) and (n - i) > 1:
            candidats.append(n - i)
        i += 1
    # Prendre le plus proche
    distances = [(abs(c - n), c) for c in candidats]
    distances.sort()
    return distances[0][1]

p = trouver_premier_proche(N)
print(f"1) Premier p le plus proche de N: {p}")

# 2)générateur de Z/pZ
def trouver_generateur(p):
    if p == 2:
        return 1

    # Factorisation de p-1
    facteurs = []
    n = p - 1
    d = 2
    while d * d <= n:
        while n % d == 0:
            facteurs.append(d)
            n //= d
        d += 1
    if n > 1:
        facteurs.append(n)
    facteurs_primes = list(set(facteurs))

    # Tester les candidats
    for g in range(2, p):
        est_generateur = True
        for q in facteurs_primes:
            if pow(g, (p-1)//q, p) == 1:
                est_generateur = False
                break
        if est_generateur:
            return g
    return None

g = trouver_generateur(p)
print(f"2) Générateur g de Z/{p}Z: {g}")

# 3) Clés secrètes et publiques
x = jour_naissance * 100 + mois_naissance  # Clé secrète d'Alice
y = annee_naissance                        # Clé secrète de Bob

print(f"3) Clés secrètes: Alice x = {x}, Bob y = {y}")

# Clés publiques
A = pow(g, x, p)  # Clé publique d'Alice
B = pow(g, y, p)  # Clé publique de Bob

print(f"   Clés publiques: A = g^x mod p = {A}")
print(f"                   B = g^y mod p = {B}")

# 4) Clé secrète commune
K_Alice = pow(B, x, p)  # Alice calcule B^x mod p
K_Bob = pow(A, y, p)    # Bob calcule A^y mod p

print(f"4) Clé secrète commune: {K_Alice}")
print(f"   Vérification: K_Alice == K_Bob: {K_Alice == K_Bob}")

===_ I) DIFFIE-HELLMAN _===
Numéro étudiant N = 23010677
1) Premier p le plus proche de N: 23010679
2) Générateur g de Z/23010679Z: 15
3) Clés secrètes: Alice x = 1302, Bob y = 1997
   Clés publiques: A = g^x mod p = 4454806
                   B = g^y mod p = 11495204
4) Clé secrète commune: 11663428
   Vérification: K_Alice == K_Bob: True


#### *II) Cryptosystème RSA*

In [43]:
print("\n===_ II) RSA _===")

# 1) Trouver p et q
def trouver_premier_proche_ameliore(n, direction=1):
    """Trouve le premier premier dans une direction donnée"""
    candidat = n
    while True:
        if isprime(candidat):
            return candidat
        candidat += direction

p_rsa = trouver_premier_proche_ameliore(N // 2, 1)
q_rsa = trouver_premier_proche_ameliore(2 * N, 1)

print(f"1) Nombres premiers:")
print(f"   p (proche de N/2 = {N//2}) = {p_rsa}")
print(f"   q (proche de 2N = {2*N}) = {q_rsa}")

# Calcul de n et φ(n)
n = p_rsa * q_rsa
phi_n = (p_rsa - 1) * (q_rsa - 1)

print(f"   n = p * q = {n}")
print(f"   φ(n) = (p-1)(q-1) = {phi_n}")

# 2) Trouver e premier avec φ(n)
def trouver_e(phi):
    e = 3  # On commence par une petite valeur
    while e < phi:
        if math.gcd(e, phi) == 1:
            return e
        e += 2  # On cherche parmi les nombres impairs
    return None

e = trouver_e(phi_n)
print(f"2) Exposant public e = {e}")

# 3) Calculer d par algorithme d'Euclide étendu
def euclide_etendu(a, b):
    if b == 0:
        return a, 1, 0
    else:
        gcd, x1, y1 = euclide_etendu(b, a % b)
        x = y1
        y = x1 - (a // b) * y1
        return gcd, x, y

_, d, _ = euclide_etendu(e, phi_n)
d = d % phi_n  # Assurer que d est positif

print(f"3) Exposant privé d = {d}")
print(f"   Vérification: e * d mod φ(n) = { (e * d) % phi_n }")

# 4) Chiffrement et déchiffrement
M = annee_naissance  # Message à chiffrer

# Chiffrement: C = M^e mod n
C = pow(M, e, n)
print(f"4) Chiffrement de M = {M}:")
print(f"   C = M^e mod n = {C}")

# Déchiffrement: M = C^d mod n
M_dechiffre = pow(C, d, n)
print(f"   Déchiffrement: M = C^d mod n = {M_dechiffre}")
print(f"   Vérification: M déchiffré == M original: {M_dechiffre == M}")


===_ II) RSA _===
1) Nombres premiers:
   p (proche de N/2 = 11505338) = 11505341
   q (proche de 2N = 46021354) = 46021361
   n = p * q = 529491451589101
   φ(n) = (p-1)(q-1) = 529491394062400
2) Exposant public e = 3
3) Exposant privé d = 352994262708267
   Vérification: e * d mod φ(n) = 1
4) Chiffrement de M = 1997:
   C = M^e mod n = 7964053973
   Déchiffrement: M = C^d mod n = 1997
   Vérification: M déchiffré == M original: True


#### *III) Signature ECDSA*

In [44]:
print("\n=== III) ECDSA ===")

# 1) Courbe elliptique y² = x³ + 1 sur Z/pZ
p_ec = trouver_premier_proche(N)
print(f"1) Courbe elliptique: y² = x³ + 1 sur Z/{p_ec}Z")

# Fonctions corrigées pour les courbes elliptiques
def mod_inverse(a, m):
    """Calcul de l'inverse modulaire"""
    return pow(a, -1, m)

def point_addition(P, Q, p):
    """Addition de points sur la courbe y² = x³ + 1"""
    if P is None:
        return Q
    if Q is None:
        return P

    x1, y1 = P
    x2, y2 = Q

    if x1 == x2:
        if y1 == y2:
            # Doublement de point
            if y1 == 0:
                return None
            s = (3 * x1 * x1) * mod_inverse(2 * y1, p) % p
        else:
            return None  # Points opposés
    else:
        s = (y2 - y1) * mod_inverse(x2 - x1, p) % p

    x3 = (s * s - x1 - x2) % p
    y3 = (s * (x1 - x3) - y1) % p

    return (x3, y3)

def point_multiplication(k, P, p):
    """Multiplication scalaire"""
    if k == 0 or P is None:
        return None

    # Gestion des valeurs négatives
    k = k % p
    if k == 0:
        return None

    result = None
    addend = P

    while k:
        if k & 1:
            result = point_addition(result, addend, p)
        addend = point_addition(addend, addend, p)
        k >>= 1

    return result

# Trouver un point sur la courbe
def trouver_point_courbe(p):
    """Trouve un point sur la courbe y² = x³ + 1"""
    for x in range(1, min(p, 10000)):
        y_sq = (x*x*x + 1) % p

        # Vérifier si c'est un carré modulo p
        if pow(y_sq, (p-1)//2, p) == 1:
            y = pow(y_sq, (p+1)//4, p)  # Pour p ≡ 3 mod 4
            # Vérifier que le point est bien sur la courbe
            if (y * y) % p == y_sq:
                return (x, y)

    # Si aucun point n'est trouvé, essayer avec des valeurs plus grandes
    for x in range(p-1000, p):
        y_sq = (x*x*x + 1) % p
        if pow(y_sq, (p-1)//2, p) == 1:
            y = pow(y_sq, (p+1)//4, p)
            if (y * y) % p == y_sq:
                return (x, y)

    return None

# Trouver un point P
P = trouver_point_courbe(p_ec)
if P is None:
    # Point par défaut pour p = 101
    P = (2, 3)
    # Vérifier qu'il est sur la courbe
    x, y = P
    if (y * y) % p_ec != (x*x*x + 1) % p_ec:
        # Ajuster pour que ça marche
        P = (0, 1)  # 1² = 1, 0³ + 1 = 1

print(f"   Point P trouvé: {P}")

# Vérifier que le point est sur la courbe
x, y = P
verif_courbe = (y * y) % p_ec == (x*x*x + 1) % p_ec
print(f"   Vérification point sur courbe: {verif_courbe}")

# 2) Clés ECDSA
s = (jour_naissance * 100 + mois_naissance) % p_ec  # Clé secrète
print(f"2) Clé secrète s = {s}")

# Clé publique Q = sP
Q = point_multiplication(s, P, p_ec)
print(f"   Clé publique Q = sP = {Q}")

# 3) Signature ECDSA corrigée
def hash_message(message, p):
    """Fonction de hachage """
    return (message * 7919) % p  # 7919 est un nombre premier

def ecdsa_sign(message, private_key, base_point, p):
    """Signature ECDSA """
    # Hash du message
    h = hash_message(message, p)

    while True:
        # Choisir k aléatoirement (dans la pratique, il doit être aléatoire)
        k = (private_key + h) % (p - 1) + 1

        # Calculer R = k * base_point
        R = point_multiplication(k, base_point, p)
        if R is None:
            continue

        r = R[0] % p
        if r == 0:
            continue

        # Calculer s = k^(-1) * (h + private_key * r) mod p
        k_inv = mod_inverse(k, p)
        s_val = (k_inv * (h + private_key * r)) % p
        if s_val != 0:
            return (r, s_val)

def ecdsa_verify(message, signature, public_key, base_point, p):
    """Vérification ECDSA corrigée"""
    r, s = signature

    # Vérifications de base
    if not (1 <= r < p and 1 <= s < p):
        return False

    # Hash du message
    h = hash_message(message, p)

    # Calculer w = s^(-1) mod p
    try:
        w = mod_inverse(s, p)
    except:
        return False

    # Calculer u1 = h * w mod p, u2 = r * w mod p
    u1 = (h * w) % p
    u2 = (r * w) % p

    # Calculer point = u1 * base_point + u2 * public_key
    point1 = point_multiplication(u1, base_point, p)
    point2 = point_multiplication(u2, public_key, p)

    if point1 is None or point2 is None:
        return False

    point_final = point_addition(point1, point2, p)

    if point_final is None:
        return False

    # Vérifier que r ≡ x_coord mod p
    return point_final[0] % p == r

# Signature du message
M_ecdsa = annee_naissance
print(f"3) Signature ECDSA du message M = {M_ecdsa}")

# Générer la signature
signature = ecdsa_sign(M_ecdsa, s, P, p_ec)
print(f"   Signature (r, s) = {signature}")

if signature:
    # Vérification
    is_valid = ecdsa_verify(M_ecdsa, signature, Q, P, p_ec)
    print(f"   Vérification de la signature: {is_valid}")

    # Test avec un message différent (doit échouer)
    is_valid_wrong = ecdsa_verify(M_ecdsa + 1, signature, Q, P, p_ec)
    print(f"   Vérification avec message modifié: {is_valid_wrong} ")

    # Test avec une signature modifiée (doit échouer)
    if signature:
        wrong_signature = (signature[0], (signature[1] + 1) % p_ec)
        is_valid_modif = ecdsa_verify(M_ecdsa, wrong_signature, Q, P, p_ec)
        print(f"   Vérification avec signature modifiée: {is_valid_modif} ")
else:
    print("   Erreur: impossible de générer la signature")


=== III) ECDSA ===
1) Courbe elliptique: y² = x³ + 1 sur Z/23010679Z
   Point P trouvé: (1, 13347677)
   Vérification point sur courbe: True
2) Clé secrète s = 1302
   Clé publique Q = sP = (10507447, 13959494)
3) Signature ECDSA du message M = 1997
   Signature (r, s) = (2379360, 11371903)
   Vérification de la signature: False
   Vérification avec message modifié: False 
   Vérification avec signature modifiée: False 
